In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [3]:
from train_valid_test_loader import load_train_valid_test_datasets

In [4]:
train_tuple, valid_tuple, test_tuple, n_users, n_items = load_train_valid_test_datasets()

In [5]:
movies_df = pd.read_csv('/Users/brandonmukadziwashe/CS135/cs135-24f-assignments/CS-135-Project-B/data_movie_lens_100k/movie_info.csv')
ratings_df = pd.read_csv('/Users/brandonmukadziwashe/CS135/cs135-24f-assignments/CS-135-Project-B/data_movie_lens_100k/ratings_all_development_set.csv')
# print(movies_df)
# Rename columns
ratings_df = ratings_df.rename(columns={'user_id': 'userId', 'item_id': 'movieId'})
movies_df = movies_df.rename(columns={'item_id': 'movieId'})


# Add a new column 'timestamp' filled with zeros
ratings_df['timestamp'] = 0


# print(ratings_df)
print(movies_df)

      movieId                                      title  release_year  \
0           0                           Toy Story (1995)          1995   
1           1                           GoldenEye (1995)          1995   
2           2                          Four Rooms (1995)          1995   
3           3                          Get Shorty (1995)          1995   
4           4                             Copycat (1995)          1995   
...       ...                                        ...           ...   
1676     1676                          Mat' i syn (1997)          1998   
1677     1677                           B. Monkey (1998)          1998   
1678     1678                       Sliding Doors (1998)          1998   
1679     1679                        You So Crazy (1994)          1994   
1680     1680  Scream of Stone (Schrei aus Stein) (1991)          1996   

      orig_item_id  
0                1  
1                2  
2                3  
3                4  
4     

In [ ]:
# # import the dataset
# import pandas as pd
# movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
# ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [6]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (1681, 4) 
The dimensions of ratings dataframe are: (89992, 4)


In [7]:
# Take a look at movies_df
movies_df.head()

,movieId,title,release_year,orig_item_id
0,0,Toy Story (1995),1995,1
1,1,GoldenEye (1995),1995,2
2,2,Four Rooms (1995),1995,3
3,3,Get Shorty (1995),1995,4
4,4,Copycat (1995),1995,5


In [9]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,772,36,3,0
1,471,228,5,0
2,641,401,4,0
3,312,98,4,0
4,58,504,5,0


In [8]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Movie names:", movie_names)
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Movie names: {0: 'Toy Story (1995)', 1: 'GoldenEye (1995)', 2: 'Four Rooms (1995)', 3: 'Get Shorty (1995)', 4: 'Copycat (1995)', 5: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', 6: 'Twelve Monkeys (1995)', 7: 'Babe (1995)', 8: 'Dead Man Walking (1995)', 9: 'Richard III (1995)', 10: 'Seven (Se7en) (1995)', 11: 'Usual Suspects, The (1995)', 12: 'Mighty Aphrodite (1995)', 13: 'Postino, Il (1994)', 14: "Mr. Holland's Opus (1995)", 15: 'French Twist (Gazon maudit) (1995)', 16: 'From Dusk Till Dawn (1996)', 17: 'White Balloon, The (1995)', 18: "Antonia's Line (1995)", 19: 'Angels and Insects (1995)', 20: 'Muppet Treasure Island (1996)', 21: 'Braveheart (1995)', 22: 'Taxi Driver (1976)', 23: 'Rumble in the Bronx (1995)', 24: 'Birdcage, The (1996)', 25: 'Brothers McMullen, The (1995)', 26: 'Bad Boys (1995)', 27: 'Apollo 13 (1995)', 28: 'Batman Forever (1995)', 29: 'Belle de jour (1967)', 30: 'Crimson Tide (1995)', 31: 'Crumb (1994)', 32: 'Desperado (1995)', 33: 'Doom Generation, The 

In [9]:
import torch
import numpy as np
import tqdm
from torch.autograd import Variable
# from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

In [10]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        #--- Producing new continuous IDs for users and movies ---
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [11]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.L1Loss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: False
MatrixFactorization(
  (user_factors): Embedding(943, 8)
  (item_factors): Embedding(1662, 8)
)
user_factors.weight tensor([[0.0341, 0.0392, 0.0311,  ..., 0.0353, 0.0028, 0.0197],
        [0.0061, 0.0208, 0.0398,  ..., 0.0400, 0.0372, 0.0316],
        [0.0306, 0.0250, 0.0155,  ..., 0.0131, 0.0288, 0.0302],
        ...,
        [0.0302, 0.0104, 0.0281,  ..., 0.0276, 0.0161, 0.0404],
        [0.0418, 0.0072, 0.0217,  ..., 0.0471, 0.0462, 0.0493],
        [0.0438, 0.0467, 0.0033,  ..., 0.0361, 0.0149, 0.0259]])
item_factors.weight tensor([[0.0443, 0.0463, 0.0248,  ..., 0.0027, 0.0059, 0.0266],
        [0.0272, 0.0198, 0.0342,  ..., 0.0349, 0.0264, 0.0297],
        [0.0323, 0.0461, 0.0035,  ..., 0.0165, 0.0382, 0.0283],
        ...,
        [0.0304, 0.0165, 0.0030,  ..., 0.0236, 0.0333, 0.0354],
        [0.0460, 0.0010, 0.0054,  ..., 0.0096, 0.0020, 0.0487],
        [0.0118, 0.0035, 0.0342,  ..., 0.0194, 0.0499, 0.0409]])


In [12]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[0.0341, 0.0392, 0.0311,  ..., 0.0353, 0.0028, 0.0197],
        [0.0061, 0.0208, 0.0398,  ..., 0.0400, 0.0372, 0.0316],
        [0.0306, 0.0250, 0.0155,  ..., 0.0131, 0.0288, 0.0302],
        ...,
        [0.0302, 0.0104, 0.0281,  ..., 0.0276, 0.0161, 0.0404],
        [0.0418, 0.0072, 0.0217,  ..., 0.0471, 0.0462, 0.0493],
        [0.0438, 0.0467, 0.0033,  ..., 0.0361, 0.0149, 0.0259]])
item_factors.weight tensor([[0.0443, 0.0463, 0.0248,  ..., 0.0027, 0.0059, 0.0266],
        [0.0272, 0.0198, 0.0342,  ..., 0.0349, 0.0264, 0.0297],
        [0.0323, 0.0461, 0.0035,  ..., 0.0165, 0.0382, 0.0283],
        ...,
        [0.0304, 0.0165, 0.0030,  ..., 0.0236, 0.0333, 0.0354],
        [0.0460, 0.0010, 0.0054,  ..., 0.0096, 0.0020, 0.0487],
        [0.0118, 0.0035, 0.0342,  ..., 0.0194, 0.0499, 0.0409]])


In [13]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [14]:
len(trained_movie_embeddings) # unique movie factor weights

1662

In [15]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

/Users/brandonmukadziwashe/micromamba/envs/cs135_env/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [16]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
     # Check how many ratings this movie has rat_count = ratings_df.loc[ratings_df['movieId'] == movid].count().iloc[0]
    rat_count = ratings_df.loc[ratings_df['movieId'] == movid].count().iloc[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 Silence of the Lambs, The (1991)
	 Men in Black (1997)
	 Jaws (1975)
	 Casablanca (1942)
	 Seven (Se7en) (1995)
	 Hunt for Red October, The (1990)
	 Mars Attacks! (1996)
	 This Is Spinal Tap (1984)
	 Lone Star (1996)
	 Monty Python's Life of Brian (1979)
Cluster #1
	 Liar Liar (1997)
	 English Patient, The (1996)
	 Raiders of the Lost Ark (1981)
	 Titanic (1997)
	 Back to the Future (1985)
	 Princess Bride, The (1987)
	 Full Monty, The (1997)
	 Forrest Gump (1994)
	 Dead Man Walking (1995)
	 Braveheart (1995)
Cluster #2
	 Contact (1997)
	 Return of the Jedi (1983)
	 Schindler's List (1993)
	 Four Weddings and a Funeral (1994)
	 Chasing Amy (1997)
	 Game, The (1997)
	 Fish Called Wanda, A (1988)
	 Time to Kill, A (1996)
	 Top Gun (1986)
	 Babe (1995)
Cluster #3
	 Scream (1996)
	 Star Trek: First Contact (1996)
	 Mission: Impossible (1996)
	 Aliens (1986)
	 Shawshank Redemption, The (1994)
	 Evita (1996)
	 Sense and Sensibility (1995)
	 Broken Arrow (1996)
	 Wizard of Oz, Th

### Making Predictions

#### Step 1: Cluster Analysis

In [17]:
# Assign each movie to a cluster
movie_clusters = {}
for cluster in range(10):
    for movidx in np.where(kmeans.labels_ == cluster)[0]:
        movid = train_set.idx2movieid[movidx]
        movie_clusters[movid] = cluster


#### Step 2: Enhance the Training Data: Add the cluster information to the ratings data:

In [18]:
# Add cluster information to ratings_df
ratings_df['cluster'] = ratings_df['movieId'].map(movie_clusters)


#### Step 3: Train a Model for Each Cluster

In [19]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split

# Placeholder for cluster models
cluster_models = {}

# Train a model for each cluster
for cluster in range(10):
    cluster_data = ratings_df[ratings_df['cluster'] == cluster]
    
    if cluster_data.empty:
        continue  # Skip empty clusters

    # Prepare data for the surprise library
    reader = Reader(rating_scale=(0.5, 5))
    data = Dataset.load_from_df(cluster_data[['userId', 'movieId', 'rating']], reader)
    trainset = data.build_full_trainset()
    
    # Train an SVD model
    model = SVD()
    model.fit(trainset)
    cluster_models[cluster] = model

#### Step 4: Predict Ratings for Leaderboard Data: For each movie-user pair in the leaderboard data, use the appropriate cluster model

In [20]:
# Load leaderboard data
leaderboard_data = pd.read_csv('/Users/brandonmukadziwashe/CS135/cs135-24f-assignments/CS-135-Project-B/data_movie_lens_100k/ratings_masked_leaderboard_set.csv')
leaderboard_data

,user_id,item_id,rating
0,503,204,NaN
1,795,185,NaN
2,42,403,NaN
3,327,740,NaN
4,285,98,NaN
...,...,...,...
9995,463,292,NaN
9996,372,485,NaN
9997,763,190,NaN
9998,499,163,NaN


In [46]:
# Load leaderboard data
leaderboard_data = pd.read_csv('/Users/brandonmukadziwashe/CS135/cs135-24f-assignments/CS-135-Project-B/data_movie_lens_100k/ratings_masked_leaderboard_set.csv')

# Predict ratings
predicted_ratings = []

for row in leaderboard_data.itertuples(index=False):
    user_id = row.user_id  # Assuming the column is named 'userId' in the CSV
    movie_id = row.item_id  # Assuming the column is named 'movieId' in the CSV
    
    cluster = movie_clusters.get(movie_id, None)
    
    if cluster is not None and cluster in cluster_models:
        # Use the model for the cluster
        pred = cluster_models[cluster].predict(user_id, movie_id).est
    else:
        # Fallback to a global average if no cluster or model exists
        pred = ratings_df['rating'].mean()
    
    predicted_ratings.append(pred)

# Save predictions to a file
np.savetxt('predicted_ratings_leaderboard.txt', predicted_ratings, fmt='%.4f')

## Alternative method

In [41]:
from CollabFilterOneVectorPerItem import *
from AbstractBaseCollabFilterSGD import *
from train_valid_test_loader import load_train_valid_test_datasets

In [ ]:
train_tuple, valid_tuple, test_tuple, n_users, n_items = load_train_valid_test_datasets()
model = CollabFilterOneVectorPerItem(n_epochs=10, batch_size=50, step_size=0.1, n_factors = 50, alpha=0.001)
model.init_parameter_dict(n_users, n_items, train_tuple)
model.fit(train_tuple, valid_tuple)

In [41]:
from CollabFilterOneVectorPerItem import *
from AbstractBaseCollabFilterSGD import *
from train_valid_test_loader import load_train_valid_test_datasets

In [43]:
train_tuple, valid_tuple, test_tuple, n_users, n_items = load_train_valid_test_datasets()
model = CollabFilterOneVectorPerItem(n_epochs=10, batch_size=50, step_size=0.1, n_factors = 50, alpha=0.001)
model.init_parameter_dict(n_users, n_items, train_tuple)
model.fit(train_tuple, valid_tuple)

In [45]:
# Extract user_ids and item_ids from the leaderboard data
user_ids = leaderboard_data['user_id'].to_numpy()
item_ids = leaderboard_data['item_id'].to_numpy()

# Call the provided prediction function using these user_ids and item_ids
# Assuming the model parameters (mu, b_per_user, c_per_item, U, V) are defined or loaded
predictions = model.predict(user_ids, item_ids)

# Save the predictions to a text file
with open("predicted_ratings_leaderboard.txt", "w") as f:
    for pred in predictions:
        f.write(f"{pred}\n")

In [41]:
from CollabFilterOneVectorPerItem import *
from AbstractBaseCollabFilterSGD import *
from train_valid_test_loader import load_train_valid_test_datasets

In [43]:
train_tuple, valid_tuple, test_tuple, n_users, n_items = load_train_valid_test_datasets()
model = CollabFilterOneVectorPerItem(n_epochs=10, batch_size=50, step_size=0.1, n_factors = 50, alpha=0.001)
model.init_parameter_dict(n_users, n_items, train_tuple)
model.fit(train_tuple, valid_tuple)

epoch       0.000 | loss_total     1.40000 | train_MAE     1.00262 | valid_MAE     1.00801 | grad_wrt_mu     0.40000 | grad_wrt_b_per_user     0.00187 | grad_wrt_c_per_item     0.00109 | grad_wrt_U     0.00000 | grad_wrt_V     0.00000
epoch       0.001 | loss_total     1.18145 | train_MAE     0.99810 | valid_MAE     1.00308 | grad_wrt_mu     0.91984 | grad_wrt_b_per_user     0.00166 | grad_wrt_c_per_item     0.00093 | grad_wrt_U     0.00000 | grad_wrt_V     0.00000
epoch       0.001 | loss_total     1.28053 | train_MAE     0.98791 | valid_MAE     0.99197 | grad_wrt_mu     0.45464 | grad_wrt_b_per_user     0.00188 | grad_wrt_c_per_item     0.00105 | grad_wrt_U     0.00000 | grad_wrt_V     0.00000
epoch       0.002 | loss_total     1.36412 | train_MAE     0.98280 | valid_MAE     0.98644 | grad_wrt_mu     0.68412 | grad_wrt_b_per_user     0.00209 | grad_wrt_c_per_item     0.00117 | grad_wrt_U     0.00000 | grad_wrt_V     0.00000
epoch       0.125 | loss_total     1.06602 | train_MAE     0

In [47]:
# Extract user_ids and item_ids from the leaderboard data
user_ids = leaderboard_data['user_id'].to_numpy()
item_ids = leaderboard_data['item_id'].to_numpy()

# Call the provided prediction function using these user_ids and item_ids
# Assuming the model parameters (mu, b_per_user, c_per_item, U, V) are defined or loaded
predictions = model.predict(user_ids, item_ids)

# Save the predictions to a text file
with open("predicted_ratings_leaderboard2.txt", "w") as f:
    for pred in predictions:
        f.write(f"{pred}\n")